In [1]:
language = 'pt'

In [ ]:
# =========================
# IMPORTS
# =========================
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# =========================
# JAVASCRIPT PARA GRAVAÇÃO
# =========================
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));

const b2text = blob => new Promise(resolve => {
    const reader = new FileReader();
    reader.onloadend = e => resolve(e.target.result);
    reader.readAsDataURL(blob);
});

var record = time => new Promise(async resolve => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const recorder = new MediaRecorder(stream);
    const chunks = [];

    recorder.ondataavailable = e => chunks.push(e.data);
    recorder.start();

    await sleep(time);

    recorder.onstop = async () => {
        const blob = new Blob(chunks);
        const text = await b2text(blob);
        resolve(text);
    };

    recorder.stop();
});
"""

# =========================
# FUNÇÃO PYTHON
# =========================
def record(sec=5):
    display(Javascript(RECORD))
    js_result = output.eval_js(f"record({sec * 1000})")
    audio = b64decode(js_result.split(',')[1])

    file_name = "request_audio.wav"
    with open(file_name, "wb") as f:
        f.write(audio)

    return file_name

# =========================
# EXECUÇÃO
# =========================
print("OUVINDO....\n")
record_file = record(5)
display(Audio(record_file, autoplay=True))


<IPython.core.display.Javascript object>

AttributeError: 'ZMQInteractiveShell' object has no attribute 'eval_js'

In [ ]:
pip install git+https://github.com/openai/whisper.git -q

In [ ]:
import whisper

model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

In [ ]:
pip install openai==0.28

In [ ]:
import openai

openai.api_key = '*******'
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[ {"role": "user", "content": transcription} ]
)

chatgpt_response = response.choices[0].message.content
print(chatgpt_response)

In [ ]:
pip install gTTS

In [ ]:
from gtts import gTTS

gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)
display(Audio(response_audio, autoplay=True))